In [83]:
import pandas as pd

In [84]:
# Load all

# OECD HCQI // Healthcare Quality Indicators
hcqi = pd.read_csv("../OECD/HEALTH_HCQI_22092023205825625.csv")

# OECD LVNG // Non-Medical Determinants of Health
lvng = pd.read_csv("../OECD/HEALTH_LVNG_25092023023230295.csv")

# OECD STAT // Health Statistics
stat = pd.read_csv("../OECD/HEALTH_STAT_25092023023520080.csv")

# OECD PROT // Social Protection
prot = pd.read_csv("../OECD/HEALTH_PROT_22092023205920251.csv")

# OECD DEMR // Demographics
demr = pd.read_csv("../OECD/HEALTH_DEMR_25092023023656863.csv")

# OECD ECOR // Economic References
ecor = pd.read_csv("../OECD/HEALTH_ECOR_28092023052908229.csv")

oecd_datasets = [hcqi, lvng, stat, prot, demr, ecor]

In [85]:
# Preliminary Cleaning

def clean_oecd_df(df):
    """Removes redundant or unneeded columns from OECD datasets"""

    df.drop(['Flag Codes', 'Flags'], axis=1, inplace=True)
    if 'YEA' in df.columns:
        df.drop(['YEA'], axis=1, inplace=True)
    else:
        df.drop(['PER'], axis=1, inplace=True)

    return df

oecd_datasets = [clean_oecd_df(x) for x in oecd_datasets]

# Renaming columns in HCQI to match other datasets
hcqi.rename(columns={'Indicator': 'Variable', 'IND': 'VAR', 'Periods': 'Year', 'Value': 'Measure'}, inplace=True)

hcqi.head()

,COU,Country,Year,VAR,Variable,GEN,Gender,AGE,Age Group,VAL,Measure,Value.1
0,AUS,Australia,2011,ADMRASTH,Asthma hospital admission,M,Male,TOTAL_15,15 years old and over,AS_STD_RATE_MPOP,Age-sex standardised rate per 100 000 population,40.4
1,AUS,Australia,2011,ADMRASTH,Asthma hospital admission,M,Male,TOTAL_15,15 years old and over,LOW_CI,Lower confidence interval,39.1
2,AUS,Australia,2011,ADMRASTH,Asthma hospital admission,M,Male,TOTAL_15,15 years old and over,UP_CI,Upper confidence interval,41.8
3,AUS,Australia,2012,ADMRASTH,Asthma hospital admission,M,Male,TOTAL_15,15 years old and over,AS_STD_RATE_MPOP,Age-sex standardised rate per 100 000 population,36.8
4,AUS,Australia,2012,ADMRASTH,Asthma hospital admission,M,Male,TOTAL_15,15 years old and over,LOW_CI,Lower confidence interval,35.5


In [86]:
# Reformatting HCQI data for merging

# Rename rows with more info
hcqi['Measure'] = hcqi['Gender'] + ', ' + hcqi['Age Group'] + ', ' + hcqi['Measure']
hcqi['VAR'] = hcqi['VAR'] + '_' + hcqi['GEN'] + '_' + hcqi['AGE'] + '_' + hcqi['VAL']
hcqi = hcqi[['VAR', 'Variable', 'Measure']]

oecd_datasets[0] = hcqi

hcqi.head()

,VAR,Variable,Measure
0,ADMRASTH_M_TOTAL_15_AS_STD_RATE_MPOP,Asthma hospital admission,"Male, 15 years old and over, Age-sex standardi..."
1,ADMRASTH_M_TOTAL_15_LOW_CI,Asthma hospital admission,"Male, 15 years old and over, Lower confidence ..."
2,ADMRASTH_M_TOTAL_15_UP_CI,Asthma hospital admission,"Male, 15 years old and over, Upper confidence ..."
3,ADMRASTH_M_TOTAL_15_AS_STD_RATE_MPOP,Asthma hospital admission,"Male, 15 years old and over, Age-sex standardi..."
4,ADMRASTH_M_TOTAL_15_LOW_CI,Asthma hospital admission,"Male, 15 years old and over, Lower confidence ..."


In [87]:
df = pd.concat(oecd_datasets[:])[['VAR', 'Variable', 'Measure']].drop_duplicates() # Merge and remove duplicate rows
# match it to look like World Bank one and so the same functions can work for the World Bank or OECD
df.columns = ['indicator_code', 'indicator_name', 'definition_or_additional_info'] # Rename to match format

df.head()

,indicator_code,indicator_name,long_definition
0,ADMRASTH_M_TOTAL_15_AS_STD_RATE_MPOP,Asthma hospital admission,"Male, 15 years old and over, Age-sex standardi..."
1,ADMRASTH_M_TOTAL_15_LOW_CI,Asthma hospital admission,"Male, 15 years old and over, Lower confidence ..."
2,ADMRASTH_M_TOTAL_15_UP_CI,Asthma hospital admission,"Male, 15 years old and over, Upper confidence ..."
6,ADMRCOPD_M_TOTAL_15_AS_STD_RATE_MPOP,Chronic obstructive pulmonary disease hospital...,"Male, 15 years old and over, Age-sex standardi..."
7,ADMRCOPD_M_TOTAL_15_LOW_CI,Chronic obstructive pulmonary disease hospital...,"Male, 15 years old and over, Lower confidence ..."


In [88]:
df.to_csv('../OECD/Cleaned/OECD_Indicator_Definition_Info.csv', index=False)